In [13]:
#r "nuget: Jinaga, 0.12.5"
#r "nuget: Jinaga.Notebooks, 0.12.5"

Installed Packages Jinaga, 0.12.5 Jinaga.Notebooks, 0.12.5

In [14]:
using Jinaga;
using Jinaga.Http;
using Jinaga.Notebooks;
using Jinaga.Notebooks.Authentication;

Create a file called `settings.json` and put the following settings into it:

```json
{
    "HttpEndpoint": "https://rep.jinaga.com/....",
    "AuthorizationToken": "eyJh...."
}
```

In [15]:
#!value --from-file settings.json --name settings

In [16]:
#!share settings --from value

var parsedSettings = System.Text.Json.JsonSerializer.Deserialize<Dictionary<string, string>>(settings);

var jinagaClient = JinagaClient.Create(options =>
{
    options.HttpEndpoint = new Uri(parsedSettings["HttpEndpoint"]);
    options.HttpAuthenticationProvider = new TokenAuthenticationProvider(parsedSettings["AuthorizationToken"]);
});

In [17]:
[FactType("Blog.User.Name")]
public record UserName(User user, string value, UserName[] prior) { }

[FactType("Blog.Site")]
public record Site(User creator, DateTime createdAt) { }

[FactType("Blog.Site.Deleted")]
public record SiteDeleted(Site site, DateTime deletedAt) { }

[FactType("Blog.Site.Restored")]
public record SiteRestored(SiteDeleted deleted) { }

[FactType("Blog.Site.Name")]
public record SiteName(Site site, string value, SiteName[] prior) { }

[FactType("Blog.Site.Domain")]
public record SiteDomain(Site site, string value, SiteDomain[] prior) { }

[FactType("Blog.Post")]
public record Post(Site site, User author, DateTime createdAt) { }

[FactType("Blog.Post.Title")]
public record PostTitle(Post post, string value, PostTitle[] prior) { }

[FactType("Blog.Post.Deleted")]
public record PostDeleted(Post post, DateTime deletedAt) { }

[FactType("Blog.Post.Restored")]
public record PostRestored(PostDeleted deleted) { }

[FactType("Blog.Post.Publish")]
public record Publish(Post post, DateTime date) { }

In [18]:
Renderer.RenderTypes(typeof(Site), typeof(SiteName), typeof(SiteDomain), typeof(SiteDeleted), typeof(SiteRestored))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Blog.Site -->
 
 Blog.Site 
 
 Blog.Site 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- Blog.Site->Jinaga.User -->
 
 Blog.Site->Jinaga.User 
 
 
 creator 
 
<!-- Blog.Site.Name -->
 
 Blog.Site.Name 
 
 Blog.Site.Name 
 
<!-- Blog.Site.Name->Blog.Site -->
 
 Blog.Site.Name->Blog.Site 
 
 
 site 
 
<!-- Blog.Site.Name->Blog.Site.Name -->
 
 Blog.Site.Name->Blog.Site.Name 
 
 
 prior 
 
<!-- Blog.Site.Domain -->
 
 Blog.Site.Domain 
 
 Blog.Site.Domain 
 
<!-- Blog.Site.Domain->Blog.Site -->
 
 Blog.Site.Domain->Blog.Site 
 
 
 site 
 
<!-- Blog.Site.Domain->Blog.Site.Domain -->
 
 Blog.Site.Domain->Blog.Site.Domain 
 
 
 prior 
 
<!-- Blog.Site.Deleted -->
 
 Blog.Site.Deleted 
 
 Blog.Site.Deleted 
 
<!-- Blog.Site.Deleted->Blog.Site -->
 
 Blog.Site.Deleted->Blog.Site 
 
 
 site 
 
<!-- Blog.Site.Restored -->
 
 Blog.Site.Restored 
 
 Blog.Site.Restored 
 
<!-- Blog.Site.Restored->Blog.Site.Deleted -->
 
 Blog.Site.Restored->Blog.Site.Deleted 
 
 
 deleted

In [19]:
var sitesForUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user &&
        !facts.Any<SiteDeleted>(deleted => deleted.site == site &&
            !facts.Any<SiteRestored>(restored => restored.deleted == deleted))
    select new
    {
        id = jinagaClient.Hash(site),
        names =
            from name in facts.OfType<SiteName>()
            where name.site == site &&
                !facts.Any<SiteName>(next => next.prior.Contains(name))
            select name.value,
        domains =
            from domain in facts.OfType<SiteDomain>()
            where domain.site == site &&
                !facts.Any<SiteDomain>(next => next.prior.Contains(domain))
            select domain.value
    }
);

(var user, var profile) = await jinagaClient.Login();
var sites = await jinagaClient.Query(sitesForUser, user);

sites

index value 0 { id = Evo8ZCV+mJKMOu1MGAfCmdxdiYPOd47YOD+fNIx61Sy/vRUdLE2xnh7zZvyFDavLrTO/jgSCPGMAJ9ThPverxA==, names = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String], domains = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String] } id Evo8ZCV+mJKMOu1MGAfCmdxdiYPOd47YOD+fNIx61Sy/vRUdLE2xnh7zZvyFDavLrTO/jgSCPGMAJ9ThPverxA== names [ Michael Perry .NET ] (values) [ Michael Perry .NET ] domains [ michaelperry.net ] (values) [ michaelperry.net ] 1 { id = s0xTUYk76BYV3FbO5Gr52x1EKHUD4E+ws8tcDSDlHaU0r8U7Zhbm+YW3NrewClKfCGYDS+82FNBPG+F8+wrqHg==, names = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String], domains = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String] } id s0xTUYk76BYV3FbO5Gr52x1EKHUD4E+ws8tcDSDlHaU0r8U7Zhbm+YW3NrewClKfCGYDS+82FNBPG+F8+wrqHg== names [ QED Code ] (values) [ QED Code ] domains [ qedcode.com ] (values) [ qedcode.com ]

In [20]:
user.publicKey

-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA8KtSvUewhXtqj6lQXod9
8WCu6UZXuSlBfc0ybWYuqDvV1dQAwxUkmP9OTYRFyBqymbFnsV1y1kY6q2k+Gi9H
Vp6Xxn7GhlusQZPWLdZfbbNsbYELh+BW8tYqRL4xn3jwZAQd7iJQLlrxY5bzqo90
wTqrLrUJe5mUZay8ACJCZkcnjrPo62A1sZVyl1LTaqDz0HqIHddgPGokjJw/nypy
vp25Cc9dlB57xKzZKt6+qdQb+tyn2eok5up7JHJFgwY+Y5i5cmgEWGhCtRz9xUQu
QZysJfNB3kWHGM79zLbGRkcw9KKy365IuFZ/2ZArvrXtZKfB92CtAKetHwsV303H
oQIDAQAB
-----END PUBLIC KEY-----


In [21]:
user.publicKey.Substring(user.publicKey.Length - 8, 6)

Y-----

In [22]:
if (false)
{
    var michaelperrynet = await jinagaClient.Fact(new Site(user, DateTime.UtcNow));
    var michaelperrynetName = await jinagaClient.Fact(new SiteName(michaelperrynet, "Michael Perry .NET", new SiteName[0]));
    var michaelperrynetDomain = await jinagaClient.Fact(new SiteDomain(michaelperrynet, "michaelperry.net", new SiteDomain[0]));

    var qedcode = await jinagaClient.Fact(new Site(user, DateTime.UtcNow.AddMilliseconds(1)));
    var qedcodeName = await jinagaClient.Fact(new SiteName(qedcode, "QED Code", new SiteName[0]));
    var qedcodeDomain = await jinagaClient.Fact(new SiteDomain(qedcode, "qedcode.com", new SiteDomain[0]));
}

In [23]:
var siteFactsForUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user &&
        !facts.Any<SiteDeleted>(deleted => deleted.site == site &&
            !facts.Any<SiteRestored>(restored => restored.deleted == deleted))
    select site
);

var siteNamesForUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user &&
        !facts.Any<SiteDeleted>(deleted => deleted.site == site &&
            !facts.Any<SiteRestored>(restored => restored.deleted == deleted))
    from name in facts.OfType<SiteName>()
    where name.site == site &&
        !facts.Any<SiteName>(next => next.prior.Contains(name))
    select name
);

var siteDomainsForUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user &&
        !facts.Any<SiteDeleted>(deleted => deleted.site == site &&
            !facts.Any<SiteRestored>(restored => restored.deleted == deleted))
    from domain in facts.OfType<SiteDomain>()
    where domain.site == site &&
        !facts.Any<SiteDomain>(next => next.prior.Contains(domain))
    select domain
);

var siteFacts = await jinagaClient.Query(siteFactsForUser, user);
var siteNames = await jinagaClient.Query(siteNamesForUser, user);
var siteDomains = await jinagaClient.Query(siteDomainsForUser, user);

jinagaClient.RenderFacts(siteFacts, siteNames, siteDomains)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 2ElTRU4NhykfzDZjVr6tXQmGG9IlrEKlFxtVPOoGHz1yQkwHtAUDLRIqJRYzBFZdVUd49lcxIHmOtJd6mwKfbQ== -->
 
 2ElTRU4NhykfzDZjVr6tXQmGG9IlrEKlFxtVPOoGHz1yQkwHtAUDLRIqJRYzBFZdVUd49lcxIHmOtJd6mwKfbQ== 
 
 Jinaga.User 
 
 publicKey 
 
 -----BEGIN PUBLIC KE... 
 
<!-- Evo8ZCV+mJKMOu1MGAfCmdxdiYPOd47YOD+fNIx61Sy/vRUdLE2xnh7zZvyFDavLrTO/jgSCPGMAJ9ThPverxA== -->
 
 Evo8ZCV+mJKMOu1MGAfCmdxdiYPOd47YOD+fNIx61Sy/vRUdLE2xnh7zZvyFDavLrTO/jgSCPGMAJ9ThPverxA== 
 
 Blog.Site 
 
 createdAt 
 
 2024-07-07T03:52:14.... 
 
 
<!-- Evo8ZCV+mJKMOu1MGAfCmdxdiYPOd47YOD+fNIx61Sy/vRUdLE2xnh7zZvyFDavLrTO/jgSCPGMAJ9ThPverxA==->2ElTRU4NhykfzDZjVr6tXQmGG9IlrEKlFxtVPOoGHz1yQkwHtAUDLRIqJRYzBFZdVUd49lcxIHmOtJd6mwKfbQ== -->
 
 Evo8ZCV+mJKMOu1MGAfCmdxdiYPOd47YOD+fNIx61Sy/vRUdLE2xnh7zZvyFDavLrTO/jgSCPGMAJ9ThPverxA==->2ElTRU4NhykfzDZjVr6tXQmGG9IlrEKlFxtVPOoGHz1yQkwHtAUDLRIqJRYzBFZdVUd49lcxIHmOtJd6mwKfbQ== 
 
 
 creator 
 
<!-- s0xTUYk76BYV3FbO5Gr52x1EKHUD4E+ws8tcDSDlHaU0r8U7Zhbm+YW3NrewClKfCGYDS+82FNBPG+F8+wrqHg== -->
 
 s0xTUYk76BYV3FbO5Gr52x1EKHUD4E+ws8tcDSDlHaU0r8U7Zhbm+YW3NrewClKfCGYDS+82FNBPG+F8+wrqHg== 
 
 Blog.Site 
 
 createdAt 
 
 2024-07-07T03:52:16.... 
 
 
<!-- s0xTUYk76BYV3FbO5Gr52x1EKHUD4E+ws8tcDSDlHaU0r8U7Zhbm+YW3NrewClKfCGYDS+82FNBPG+F8+wrqHg==->2ElTRU4NhykfzDZjVr6tXQmGG9IlrEKlFxtVPOoGHz1yQkwHtAUDLRIqJRYzBFZdVUd49lcxIHmOtJd6mwKfbQ== -->
 
 s0xTUYk76BYV3FbO5Gr52x1EKHUD4E+ws8tcDSDlHaU0r8U7Zhbm+YW3NrewClKfCGYDS+82FNBPG+F8+wrqHg==->2ElTRU4NhykfzDZjVr6tXQmGG9IlrEKlFxtVPOoGHz1yQkwHtAUDLRIqJRYzBFZdVUd49lcxIHmOtJd6mwKfbQ== 
 
 
 creator 
 
<!-- rLHVvxL/F6aBGhshWU4BL/dSBG/IjSUFvaNjIjz+EZlzMXYYVyjeVhC+7pvr+8wpY7KOd3HIqLCyjZDvq0JtGQ== -->
 
 rLHVvxL/F6aBGhshWU4BL/dSBG/IjSUFvaNjIjz+EZlzMXYYVyjeVhC+7pvr+8wpY7KOd3HIqLCyjZDvq0JtGQ== 
 
 Blog.Site.Name 
 
 value 
 
 Michael Perry .NET 
 
 
<!-- rLHVvxL/F6aBGhshWU4BL/dSBG/IjSUFvaNjIjz+EZlzMXYYVyjeVhC+7pvr+8wpY7KOd3HIqLCyjZDvq0JtGQ==->Evo8ZCV+mJKMOu1MGAfCmdxdiYPOd47YOD+fNIx61Sy/vRUdLE2xnh7zZvyFDavLrTO/jgSCPGMAJ9ThPverxA== -->
 
 rLHVvxL/F6aBGhshWU4BL/dSBG/IjSUFvaNjIjz+EZlzMXYYVyjeVhC+7pvr+8wpY7KOd3HIqLCyjZDvq0JtGQ==->Evo8ZCV+mJKMOu1MGAfCmdxdiYPOd47YOD+fNIx61Sy/vRUdLE2xnh7zZvyFDavLrTO/jgSCPGMAJ9ThPverxA== 
 
 
 site 
 
<!-- w+sBjIqFUqrViR+HZPIlhSs9K70UzIQNUawsLz++DcWvMjGMrfsvPmledtOhk6mCXlAiEiqrnLV0/vsJasRbkw== -->
 
 w+sBjIqFUqrViR+HZPIlhSs9K70UzIQNUawsLz++DcWvMjGMrfsvPmledtOhk6mCXlAiEiqrnLV0/vsJasRbkw== 
 
 Blog.Site.Name 
 
 value 
 
 QED Code 
 
 
<!-- w+sBjIqFUqrViR+HZPIlhSs9K70UzIQNUawsLz++DcWvMjGMrfsvPmledtOhk6mCXlAiEiqrnLV0/vsJasRbkw==->s0xTUYk76BYV3FbO5Gr52x1EKHUD4E+ws8tcDSDlHaU0r8U7Zhbm+YW3NrewClKfCGYDS+82FNBPG+F8+wrqHg== -->
 
 w+sBjIqFUqrViR+HZPIlhSs9K70UzIQNUawsLz++DcWvMjGMrfsvPmledtOhk6mCXlAiEiqrnLV0/vsJasRbkw==->s0xTUYk76BYV3FbO5Gr52x1EKHUD4E+ws8tcDSDlHaU0r8U7Zhbm+YW3NrewClKfCGYDS+82FNBPG+F8+wrqHg== 
 
 
 site 
 
<!-- Y1AdLNwTFiJtLJzDRvwyJcj0vwP9ccnIxuJgmKja0+1hUWjWwdroNXmyo34pQXshS0sc8Jx9wsqYbqgxplMtZg== -->
 
 Y1AdLNwTFiJtLJzDRvwyJcj0vwP9ccnIxuJgmKja0+1hUWjWwdroNXmyo34pQXshS0sc8Jx9wsqYbqgxplMtZg== 
 
 Blog.Site.Domain 
 
 value 
 
 michaelperry.net 
 
 
<!-- Y1AdLNwTFiJtLJzDRvwyJcj0vwP9ccnIxuJgmKja0+1hUWjWwdroNXmyo34pQXshS0sc8Jx9wsqYbqgxplMtZg==->Evo8ZCV+mJKMOu1MGAfCmdxdiYPOd47YOD+fNIx61Sy/vRUdLE2xnh7zZvyFDavLrTO/jgSCPGMAJ9ThPverxA== -->
 
 Y1AdLNwTFiJtLJzDRvwyJcj0vwP9ccnIxuJgmKja0+1hUWjWwdroNXmyo34pQXshS0sc8Jx9wsqYbqgxplMtZg==->Evo8ZCV+mJKMOu1MGAfCmdxdiYPOd47YOD+fNIx61Sy/vRUdLE2xnh7zZvyFDavLrTO/jgSCPGMAJ9ThPverxA== 
 
 
 site 
 
<!-- WFeqgmImIUc6HZuQ1eKl8v6t1DD9IRMHFZHrAxIJznvCRAWDxENSHJVZm3GdIOvCyJieEGGWBVD7s8nnQdmwfA== -->
 
 WFeqgmImIUc6HZuQ1eKl8v6t1DD9IRMHFZHrAxIJznvCRAWDxENSHJVZm3GdIOvCyJieEGGWBVD7s8nnQdmwfA== 
 
 Blog.Site.Domain 
 
 value 
 
 qedcode.com 
 
 
<!-- WFeqgmImIUc6HZuQ1eKl8v6t1DD9IRMHFZHrAxIJznvCRAWDxENSHJVZm3GdIOvCyJieEGGWBVD7s8nnQdmwfA==->s0xTUYk76BY